In [ ]:
import pandas as pd
import os
from IPython.display import display

# Ruta a los CSVs
base_path = r"C:\Users\rober\Desktop\BigData+\Data Analysis with Python\Proyecto 1 - Análisis exploratorio de un dataset\data\csvs_per_year\csvs_per_year"

# Listar archivos
all_files = [os.path.join(base_path, f) for f in os.listdir(base_path) if f.endswith(".csv")]

# Leer y concatenar
df_list = []
for f in all_files:
    df_temp = pd.read_csv(f, sep=";")
    
    # Normalizar nombres de columnas (quitar espacios, poner mayúsculas iguales)
    df_temp.columns = df_temp.columns.str.strip().str.upper()
    
    # Unificar el nombre de la columna de fecha
    if "DATE" in df_temp.columns:
        df_temp.rename(columns={"DATE": "date"}, inplace=True)
    
    # Añadir a la lista
    df_list.append(df_temp)

# Concatenar todo en un único DataFrame
df_full = pd.concat(df_list, ignore_index=True)

# Eliminar columnas duplicadas si las hubiera
df_full = df_full.loc[:, ~df_full.columns.duplicated()]

# Convertir la columna 'date' a datetime
df_full["date"] = pd.to_datetime(df_full["date"], errors="coerce")

# Ordenar por fecha
df_full = df_full.sort_values("date").reset_index(drop=True)

# Mostrar resultado
print("Dimensiones del dataset final:", df_full.shape)
display(df_full.head())


,"date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PXY,SO_2,TCH,TOL,station","date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PM25,PXY,SO_2,TCH,TOL,station","date,BEN,CO,EBE,NMHC,NO,NO_2,O_3,PM10,PM25,SO_2,TCH,TOL,station","date,BEN,CH4,CO,EBE,NMHC,NO,NO_2,NOx,O_3,PM10,PM25,SO_2,TCH,TOL,station"
0,"2001-08-01 01:00:00,,0.3700000047683716,,,,58....",NaN,NaN,NaN
1,"2001-08-01 01:00:00,1.5,0.3400000035762787,1.4...",NaN,NaN,NaN
2,"2001-08-01 01:00:00,,0.2800000011920929,,,,50....",NaN,NaN,NaN
3,"2001-08-01 01:00:00,,0.4699999988079071,,,,69....",NaN,NaN,NaN
4,"2001-08-01 01:00:00,,0.38999998569488525,,,,22...",NaN,NaN,NaN
